In [1]:
import pandas as pd
import numpy as np
import matplotlib

from calibUtils import *

In [2]:
df = pd.read_csv('./files/30-04-21_SN04calib_test5_DACmaxFFF.csv' ,delimiter=';', header=None)
df = df.rename(columns={0: 'ch', 1:'ADCmin', 2: 'DMMmin', 3:'ADCmax', 4:'DMMmax'})
df

,ch,ADCmin,DMMmin,ADCmax,DMMmax
0,ch0,E6DD,108.752,F6DB,116.327
1,ch1,E6BB,108.755,F6CC,116.332
2,ch2,E6B5,108.740,F6C7,116.314
3,ch3,E6AE,108.744,F6BE,116.318
4,ch4,E6C0,108.744,F6D4,116.320
5,ch5,E6B8,108.744,F6C8,116.312
6,ch6,E6AF,108.735,F6C2,116.309
7,ch7,E6B7,108.741,F6C7,116.315
8,ch8,E60D,108.745,F712,116.320
9,ch9,E6CF,108.715,F6E5,116.291


In [3]:
#convert ADCmin and ADCmax in decimal numbers
df = hex_to_dec(df)
df


,ch,ADCmin,DMMmin,ADCmax,DMMmax
0,ch0,59101,108.752,63195,116.327
1,ch1,59067,108.755,63180,116.332
2,ch2,59061,108.740,63175,116.314
3,ch3,59054,108.744,63166,116.318
4,ch4,59072,108.744,63188,116.320
5,ch5,59064,108.744,63176,116.312
6,ch6,59055,108.735,63170,116.309
7,ch7,59063,108.741,63175,116.315
8,ch8,58893,108.745,63250,116.320
9,ch9,59087,108.715,63205,116.291


In [4]:
#data for HV calibration

data_HV = {'ADCmin': [0x02EB], 'DMMmin': [1.378], 'ADCmax': [0xFEB5], 'DMMmax': [120]}
df_HV = pd.DataFrame(data=data_HV)
df_HV

,ADCmin,DMMmin,ADCmax,DMMmax
0,747,1.378,65205,120


In [5]:
# compute gain and offset for the Bias Voltages
# a_coeff = 1.8535e-3
# b_coeff = 0
# Gain BiasV = (DMMmax - DMMmin) / (ADCmax - ADCmin ) * 1/a_coeff
# offset BiasV = ADCmin - (DMMmin - b_coeff )/ (Gain BiasV * a_coeff )

df = biasV_params(df)
df

,ch,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,offset BiasV
0,ch0,59101,108.752,63195,116.327,0.998257,-324.671551
1,ch1,59067,108.755,63180,116.332,0.993908,-31.852184
2,ch2,59061,108.740,63175,116.314,0.993273,3.742540
3,ch3,59054,108.744,63166,116.318,0.993756,-15.799842
4,ch4,59072,108.744,63188,116.320,0.993052,8.029567
5,ch5,59064,108.744,63176,116.312,0.992968,21.006342
6,ch6,59055,108.735,63170,116.309,0.993031,21.383021
7,ch7,59063,108.741,63175,116.315,0.993756,-26.428571
8,ch8,58893,108.745,63250,116.320,0.937999,3655.114191
9,ch9,59087,108.715,63205,116.291,0.992570,5.973865


In [6]:
#Compute gain and offset for the HV

# a_coeff = 1.8409e-3
# b_coeff = 0
# Gain BiasV = (DMMmax - DMMmin) / (ADCmax - ADCmin ) * 1/a_coeff
# offset BiasV = ADCmin - (DMMmin - b_coeff )/ (Gain BiasV * a_coeff )


df_HV = HV_params(df_HV)
df_HV

,ADCmin,DMMmin,ADCmax,DMMmax,Gain HV,offset HV
0,747,1.378,65205,120,0.999674,1.79132


In [7]:
#convert to ADC the bias voltages 
# from page 13, ADC calib = 108.7677 + 1.8525e-3 * ADC <-- it's too high!
#df = convertADC_BiasV(df)
#df

In [8]:
#df_HV = convertADC(df_HV)  
#df_HV

In [9]:
# Gain biasV = math.ceil(Gain BiasV* 32768)
# Offset biasV = round(offset BiasV)  

df = code_offsetandgain(df,'biasV')
df

,ch,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,offset BiasV,Gain_Usig BiasV,Offset_Sig BiasV
0,ch0,59101,108.752,63195,116.327,0.998257,-324.671551,32711,-325
1,ch1,59067,108.755,63180,116.332,0.993908,-31.852184,32568,-32
2,ch2,59061,108.740,63175,116.314,0.993273,3.742540,32548,4
3,ch3,59054,108.744,63166,116.318,0.993756,-15.799842,32563,-16
4,ch4,59072,108.744,63188,116.320,0.993052,8.029567,32540,8
5,ch5,59064,108.744,63176,116.312,0.992968,21.006342,32538,21
6,ch6,59055,108.735,63170,116.309,0.993031,21.383021,32540,21
7,ch7,59063,108.741,63175,116.315,0.993756,-26.428571,32563,-26
8,ch8,58893,108.745,63250,116.320,0.937999,3655.114191,30736,3655
9,ch9,59087,108.715,63205,116.291,0.992570,5.973865,32525,6


In [10]:
# Gain HV = math.ceil(Gain HV* 32768)
# Offset HV = round(offset HV)  

df_HV = code_offsetandgain(df_HV,'HV')
df_HV

,ADCmin,DMMmin,ADCmax,DMMmax,Gain HV,offset HV,Gain_Usig HV,Offset_Sig HV
0,747,1.378,65205,120,0.999674,1.79132,32757,2


In [11]:
# convert following prescriptions
 # GainUsig BiasV   = hex(Gain_Usig BiasV) 
 # Offset_Sig BiasV = tohextwocompl(Offset_Sig BiasV ,15) 

df = convert_to_exadec(df, 'biasV')
df

,ch,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,offset BiasV,Gain_Usig BiasV,Offset_Sig BiasV,Gain_hex BiasV,Offset_hex BiasV
0,ch0,59101,108.752,63195,116.327,0.998257,-324.671551,32711,-325,0x7fc7,0xfebb
1,ch1,59067,108.755,63180,116.332,0.993908,-31.852184,32568,-32,0x7f38,0xffe0
2,ch2,59061,108.740,63175,116.314,0.993273,3.742540,32548,4,0x7f24,0x4
3,ch3,59054,108.744,63166,116.318,0.993756,-15.799842,32563,-16,0x7f33,0xfff0
4,ch4,59072,108.744,63188,116.320,0.993052,8.029567,32540,8,0x7f1c,0x8
5,ch5,59064,108.744,63176,116.312,0.992968,21.006342,32538,21,0x7f1a,0x15
6,ch6,59055,108.735,63170,116.309,0.993031,21.383021,32540,21,0x7f1c,0x15
7,ch7,59063,108.741,63175,116.315,0.993756,-26.428571,32563,-26,0x7f33,0xffe6
8,ch8,58893,108.745,63250,116.320,0.937999,3655.114191,30736,3655,0x7810,0xe47
9,ch9,59087,108.715,63205,116.291,0.992570,5.973865,32525,6,0x7f0d,0x6


In [12]:
# convert following prescriptions
# GainUsig BiasV   = hex(Gain_Usig HV) 
# Offset_Sig BiasV = tohextwocompl(Offset_Sig HV ,15) 

df_HV = convert_to_exadec(df_HV, 'HV')
df_HV

,ADCmin,DMMmin,ADCmax,DMMmax,Gain HV,offset HV,Gain_Usig HV,Offset_Sig HV,Gain_hex HV,Offset_hex HV
0,747,1.378,65205,120,0.999674,1.79132,32757,2,0x7ff5,0x2


In [13]:
df_calib_biasV = df[['Gain_hex BiasV','Offset_hex BiasV'] ].copy()
df_calib_biasV.to_csv('calibDB/calib_biasV_SN04_v5.csv')

In [14]:
df_calib_HV = df_HV[['Gain_hex HV','Offset_hex HV'] ].copy()
df_calib_HV.to_csv('calibDB/calib_HV_SN04_v5.csv')

In [15]:
df_calib_biasV

,Gain_hex BiasV,Offset_hex BiasV
0,0x7fc7,0xfebb
1,0x7f38,0xffe0
2,0x7f24,0x4
3,0x7f33,0xfff0
4,0x7f1c,0x8
5,0x7f1a,0x15
6,0x7f1c,0x15
7,0x7f33,0xffe6
8,0x7810,0xe47
9,0x7f0d,0x6


In [16]:
df_testcalib = testcalib_biasV(df)
df_testcalib

,ch,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,offset BiasV,Gain_Usig BiasV,Offset_Sig BiasV,Gain_hex BiasV,Offset_hex BiasV,ADCcalmin,ADCcalmax,ADCmin err,ADCmax err,ADCmin sigma,ADCmax sigma
0,ch0,59101,108.752,63195,116.327,0.998257,-324.671551,32711,-325,0x7fc7,0xfebb,108.751812,116.326842,0.000188,0.000158,0.101231,0.085444
1,ch1,59067,108.755,63180,116.332,0.993908,-31.852184,32568,-32,0x7f38,0xffe0,108.753517,116.330433,0.001483,0.001567,0.799869,0.845360
2,ch2,59061,108.740,63175,116.314,0.993273,3.742540,32548,4,0x7f24,0x4,108.741964,116.316067,-0.001964,-0.002067,-1.059408,-1.115386
3,ch3,59054,108.744,63166,116.318,0.993756,-15.799842,32563,-16,0x7f33,0xfff0,108.742347,116.316257,0.001653,0.001743,0.891945,0.940215
4,ch4,59072,108.744,63188,116.320,0.993052,8.029567,32540,8,0x7f1c,0x8,108.742845,116.318768,0.001155,0.001232,0.623207,0.664579
5,ch5,59064,108.744,63176,116.312,0.992968,21.006342,32538,21,0x7f1a,0x15,108.745364,116.313459,-0.001364,-0.001459,-0.735740,-0.787382
6,ch6,59055,108.735,63170,116.309,0.993031,21.383021,32540,21,0x7f1c,0x15,108.735482,116.309565,-0.000482,-0.000565,-0.260303,-0.304929
7,ch7,59063,108.741,63175,116.315,0.993756,-26.428571,32563,-26,0x7f33,0xffe6,108.740505,116.314415,0.000495,0.000585,0.267130,0.315400
8,ch8,58893,108.745,63250,116.320,0.937999,3655.114191,30736,3655,0x7810,0xe47,108.743531,116.318442,0.001469,0.001558,0.792610,0.840361
9,ch9,59087,108.715,63205,116.291,0.992570,5.973865,32525,6,0x7f0d,0x6,108.716634,116.292745,-0.001634,-0.001745,-0.881785,-0.941426
